# Случайные леса
__Суммарное количество баллов: 10__

__Решение отправлять на `ml.course.practice@gmail.com`__

__Тема письма: `[ML][MS][HW09] <ФИ>`, где вместо `<ФИ>` указаны фамилия и имя__

В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным пользователей социальной сети Вконтакте, и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

В результате мы сможем определить, какие подписки пользователей больше всего влияют на определение возраста и пола человека. 

In [1]:
!pip install catboost

     |████████████████████████████████| 65.7MB 47kB/s 


In [2]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier
from scipy.stats import mode

from typing import Callable, Union, NoReturn, Optional, Dict, Any, List

In [3]:
def gini(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return np.sum(proba * (1 - proba))
    
def entropy(x):
    _, counts = np.unique(x, return_counts=True)
    proba = counts / len(x)
    return -np.sum(proba * np.log2(proba))

def gain(left_y, right_y, criterion):
    y = np.concatenate((left_y, right_y))
    return criterion(y) - (criterion(left_y) * len(left_y) + criterion(right_y) * len(right_y)) / len(y)

### Задание 1 (2 балла)
Random Forest состоит из деревьев решений. Каждое такое дерево строится на одной из выборок, полученных при помощи bagging. Элементы, которые не вошли в новую обучающую выборку, образуют out-of-bag выборку. Кроме того, в каждом узле дерева мы случайным образом выбираем набор из `max_features` и ищем признак для предиката разбиения только в этом наборе.

Сегодня мы будем работать только с бинарными признаками, поэтому нет необходимости выбирать значение признака для разбиения.

#### Методы
`predict(X)` - возвращает предсказанные метки для элементов выборки `X`

#### Параметры конструктора
`X, y` - обучающая выборка и соответствующие ей метки классов. Из нее нужно получить выборку для построения дерева при помощи bagging. Out-of-bag выборку нужно запомнить, она понадобится потом.

`criterion="gini"` - задает критерий, который будет использоваться при построении дерева. Возможные значения: `"gini"`, `"entropy"`.

`max_depth=None` - ограничение глубины дерева. Если `None` - глубина не ограничена

`min_samples_leaf=1` - минимальное количество элементов в каждом листе дерева.

`max_features="auto"` - количество признаков, которые могут использоваться в узле. Если `"auto"` - равно `sqrt(X.shape[1])`

In [4]:
class DecisionTreeLeaf:
    """

    Attributes
    ----------
    y : dict
        Словарь, отображающий метки в вероятность того, что объект, попавший в данный лист, принадлжит классу, соответствующиему метке 
    """
    def __init__(self, x):
        self.prediction = {}
        values = np.unique(x)

        for val in values:
            self.prediction[val] = len(x[x == val]) / len(x)

        values, counts = np.unique(x, return_counts=True)
        self.y = values[np.argmax(counts)]

class DecisionTreeNode:
    """

    Attributes
    ----------
    split_dim : int
        Измерение, по которому разбиваем выборку.
    split_value : float
        Значение, по которому разбираем выборку.
    left : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] < split_value.
    right : Union[DecisionTreeNode, DecisionTreeLeaf]
        Поддерево, отвечающее за случай x[split_dim] >= split_value. 
    """
    def __init__(self, split_dim: int, split_value: float, 
                 left: Union['DecisionTreeNode', DecisionTreeLeaf], 
                 right: Union['DecisionTreeNode', DecisionTreeLeaf]):
        self.split_dim = split_dim
        self.split_value = split_value
        self.left = left
        self.right = right

In [74]:
class DecisionTree:
    def __init__(self, X, y, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto"):
        self.max_features = max_features if isinstance(max_features, int) else int(np.sqrt(X.shape[1])) + 1
        self.min_samples_leaf = min_samples_leaf if min_samples_leaf is not None else 0
        self.criterion = gini if 'gini' else entropy
        self.max_depth = max_depth if max_depth is not None else 1e+15

        N = X.shape[0]
        random_idx = np.random.choice(np.arange(N), N, replace=True)
        self.X = X[random_idx]
        self.y = y[random_idx]

        self.oob_X = X[~random_idx]
        self.oob_y = y[~random_idx]

        self.root = None

    def fit(self):
        depth = 0
        self.root = self.__split_tree(self.X, self.y, depth)

    def __split_tree(self, X, y, depth):
        depth += 1

        if len(y) > self.min_samples_leaf:
            gains = []

            if X.shape[1] > self.max_features:
                features = np.random.choice(np.arange(X.shape[1]), self.max_features, replace=False)
            else:
                features = np.arange(X.shape[1])

            for dim in features:
                gain, split_value = self.__split_tree_dim(X, y, dim)
                gains.append((gain, split_value, dim))

            best_gain, split_value, split_dim = max(gains, key=lambda x: x[0])
            
            left_mask = X[:, split_dim] < split_value
            right_mask = ~left_mask

            if len(X[left_mask]) == 0 or len(X[right_mask]) == 0 or depth > self.max_depth:
                return DecisionTreeLeaf(y)

            left = self.__split_tree(X[left_mask], y[left_mask], depth)
            right = self.__split_tree(X[right_mask], y[right_mask], depth)

            return DecisionTreeNode(split_dim, split_value, left, right)

        return DecisionTreeLeaf(y)

    def __split_tree_dim(self, X, y, split_dim):
        x_dim = X[:, split_dim]

        split_val = 0.5

        left_y, right_y = y[x_dim < split_val], y[x_dim >= split_val]
        gain_node = gain(left_y, right_y, self.criterion)

        return gain_node, split_val

    def __go_down(self, X, node):
        y_pred = np.empty(X.shape[0], dtype=np.object)

        if isinstance(node, DecisionTreeLeaf):
            return node.prediction

        left_node = X[:, node.split_dim] < node.split_value
        y_pred[left_node] = self.__go_down(X[left_node], node.left)
        y_pred[~left_node] = self.__go_down(X[~left_node], node.right)
        
        return y_pred

    def predict_proba(self, X: np.ndarray) ->  List[Dict[Any, float]]:       
        return list(self.__go_down(X, self.root))
        
    def predict(self, X) -> list:
        proba = self.predict_proba(X)
        return [max(p.keys(), key=lambda k: p[k]) for p in proba]

### Задание 2 (2 балла)
Теперь реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

#### Параметры конструктора
`n_estimators` - количество используемых для предсказания деревьев.

Остальное - параметры деревьев.

#### Методы
`fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

`predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [81]:
from tqdm.notebook import tqdm

class RandomForestClassifier:
    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.criterion = criterion

        self.trees = []
    
    def fit(self, X, y):
        for i in tqdm(range(self.n_estimators)):
            tree = DecisionTree(X, y, self.criterion, self.max_depth, self.min_samples_leaf, self.max_features)
            tree.fit()

            self.trees.append(tree)
    
    def predict(self, X):
        preds = self.trees[0].predict(X)

        for tree in self.trees[1:]:
            preds = np.vstack((preds, tree.predict(X)))
        
        return mode(preds, axis=0)[0].squeeze()

### Задание 3 (2 балла)
Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest - посчитать out-of-bag ошибку предсказания `err_oob`, а затем перемешать значения признака `j` и посчитать ее (`err_oob_j`) еще раз. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [11]:
def feature_importance(rfc):
    err_j = np.zeros((len(rfc.trees), rfc.trees[0].oob_X.shape[1]))

    for i, tree in enumerate(rfc.trees):
        oob_X, oob_y = tree.oob_X, tree.oob_y
        N, M = oob_X.shape

        err_oob = np.mean(tree.predict(oob_X) == oob_y)        

        for j in np.arange(M):
            shuffled_X = oob_X.copy()
            random_idx = np.random.choice(np.arange(N), N, replace=False)
            shuffled_X[:, j] = oob_X[:, j][random_idx]

            err_oob_j = np.mean(tree.predict(shuffled_X) == oob_y)
            err_j[i][j] = err_oob - err_oob_j

    err_j = np.mean(err_j, axis=0)
    return err_j

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]

Наконец, пришло время протестировать наше дерево на простом синтетическом наборе данных. В результате точность должна быть примерно равна `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [83]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)

print("Accuracy:", np.mean(rfc.predict(X) == y))
print("Importance:", feature_importance(rfc))


Accuracy: 1.0
Importance: [4.2000e-04 1.0000e-05 2.0176e-01 2.0637e-01 3.9501e-01 4.2000e-04]


### Задание 4 (1 балл)
Теперь поработаем с реальными данными.

Выборка состоит из публичных анонимизированных данных пользователей социальной сети Вконтакте. Первые два столбца отражают возрастную группу (`zoomer`, `doomer` и `boomer`) и пол (`female`, `male`). Все остальные столбцы являются бинарными признаками, каждый из них определяет, подписан ли пользователь на определенную группу/публичную страницу или нет.\
\
Необходимо обучить два классификатора, один из которых определяет возрастную группу, а второй - пол.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются. Лес должен строиться за какое-то разумное время.

In [5]:
def read_dataset(path):
    dataframe = pandas.read_csv(path, header=0)
    dataset = dataframe.values.tolist()
    random.shuffle(dataset)
    y_age = [row[0] for row in dataset]
    y_sex = [row[1] for row in dataset]
    X = [row[2:] for row in dataset]
    
    return np.array(X), np.array(y_age), np.array(y_sex), list(dataframe.columns)[2:]

In [85]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)

#### Возраст

In [87]:
%%time

rfc = RandomForestClassifier(n_estimators=100, max_depth=15, min_samples_leaf=10)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)


Accuracy: 0.694829760403531
Most important features:
1. mudakoff
2. ovsyanochan
3. 4ch
4. rhymes
5. styd.pozor
6. dayvinchik
7. tumblr_vacuum
8. rapnewrap
9. iwantyou
10. pixel_stickers
11. pravdashowtop
12. reflexia_our_feelings
13. leprum
14. i_des
15. bot_maxim
16. xfilm
17. pozor
18. memeboizz
19. thesmolny
20. ne.poverish
CPU times: user 13min 57s, sys: 26.9 s, total: 14min 23s
Wall time: 14min 26s


In [88]:
from sklearn import ensemble

rfc = ensemble.RandomForestClassifier(n_estimators=100)

rfc.fit(X_train, y_age_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_age_test))

Accuracy: 0.7667087011349306


#### Пол

In [90]:
rfc = RandomForestClassifier(n_estimators=100, max_depth=15, min_samples_leaf=10)
rfc.fit(X_train, y_sex_train)

print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(feature_importance(rfc), features, 20)):
    print(str(i+1) + ".", name)


Accuracy: 0.7894073139974779
Most important features:
1. 40kg
2. girlmeme
3. zerofat
4. mudakoff
5. modnailru
6. 9o_6o_9o
7. cook_good
8. i_d_t
9. be.beauty
10. sh.cook
11. femalemem
12. igm
13. thesmolny
14. be.women
15. reflexia_our_feelings
16. 4ch
17. recipes40kg
18. bon
19. soverwenstvo.decora
20. woman.blog


In [91]:
rfc = ensemble.RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_sex_train)
print("Accuracy:", np.mean(rfc.predict(X_test) == y_sex_test))

Accuracy: 0.8638083228247163


### CatBoost
В качестве аьтернативы попробуем CatBoost. 

Устаниовить его можно просто с помощью `pip install catboost`. Туториалы можно найти, например, [здесь](https://catboost.ai/docs/concepts/python-usages-examples.html#multiclassification) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb). Главное - не забудьте использовать `loss_function='MultiClass'`.\
\
Сначала протестируйте CatBoost на синтетических данных. Выведите точность и важность признаков.

In [94]:
cat_params = {
                'n_estimators':5000,
                'learning_rate': 0.01,
#                'eval_metric':'AUC',
                'loss_function':'MultiClass',
                'random_seed': 42,
                'metric_period':5000,
                'od_wait':500,
                'task_type':'CPU',
                'depth': 8,
                }

In [95]:
X, y = synthetic_dataset(1000)

clf = CatBoostClassifier(**cat_params)
clf.fit(X, y)

y_pred = clf.predict(X).reshape(1, -1)

print("Accuracy:", np.mean(y_pred == y))
print("Importance:", clf.get_feature_importance())

0:	learn: 1.0793979	total: 49.2ms	remaining: 4m 5s
4999:	learn: 0.0006090	total: 4.04s	remaining: 0us
Accuracy: 1.0
Importance: [1.11412711e-02 1.04435330e-02 2.78226570e+01 2.78208330e+01
 4.43253256e+01 9.59964946e-03]


### Задание 5 (3 балла)
Попробуем применить один из используемых на практике алгоритмов. В этом нам поможет CatBoost. Также, как и реализованный ними RandomForest, применим его для определения пола и возраста пользователей сети Вконтакте, выведите названия наиболее важных признаков так же, как в задании 3.\
\
Эксперименты с множеством используемых признаков и подбор гиперпараметров приветствуются.

In [7]:
X, y_age, y_sex, features = read_dataset("vk.csv")
X_train, X_test, y_age_train, y_age_test, y_sex_train, y_sex_test = train_test_split(X, y_age, y_sex, train_size=0.9)
X_train, X_eval, y_age_train, y_age_eval, y_sex_train, y_sex_eval = train_test_split(X_train, y_age_train, y_sex_train, train_size=0.8)

In [8]:
cat_params = {
                'n_estimators':5000,
                'learning_rate': 0.005,
                'eval_metric':'Accuracy',
                'loss_function':'MultiClass',
                'random_seed': 42,
                'metric_period':500,
                'od_wait':500,
                'task_type':'GPU',
                'depth': 10,
                }

#### Возраст

In [12]:
clf = CatBoostClassifier(**cat_params)
clf.fit(X_train, y_age_train, eval_set=(X_eval, y_age_eval),
            cat_features=np.arange(X.shape[1]),
            use_best_model=True,
            verbose=True)

y_pred = clf.predict(X_test).reshape(1, -1)

print("Accuracy:", np.mean(y_pred == y_age_test))
print("Most important features:")
for i, name in enumerate(most_important_features(clf.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

0:	learn: 0.6302595	test: 0.6033637	best: 0.6033637 (0)	total: 25.1ms	remaining: 2m 5s
500:	learn: 0.7307153	test: 0.7021724	best: 0.7021724 (498)	total: 10.6s	remaining: 1m 35s
1000:	learn: 0.7710379	test: 0.7196917	best: 0.7203924 (990)	total: 21.1s	remaining: 1m 24s
1500:	learn: 0.7994390	test: 0.7274001	best: 0.7274001 (1498)	total: 31.1s	remaining: 1m 12s
2000:	learn: 0.8199509	test: 0.7302032	best: 0.7302032 (1969)	total: 41.7s	remaining: 1m 2s
2500:	learn: 0.8392356	test: 0.7323055	best: 0.7323055 (2272)	total: 52.4s	remaining: 52.4s
3000:	learn: 0.8558906	test: 0.7316048	best: 0.7337071 (2875)	total: 1m 2s	remaining: 41.9s
3500:	learn: 0.8707924	test: 0.7365102	best: 0.7365102 (3303)	total: 1m 13s	remaining: 31.5s
4000:	learn: 0.8813114	test: 0.7379117	best: 0.7393132 (3598)	total: 1m 24s	remaining: 21.1s
bestTest = 0.7393132446
bestIteration = 3598
Shrink model to first 3599 iterations.
Accuracy: 0.7402269861286255
Most important features:
1. ovsyanochan
2. styd.pozor
3. 4ch
4

#### Пол

In [13]:
clf = CatBoostClassifier(**cat_params)
clf.fit(X_train, y_sex_train, eval_set=(X_eval, y_sex_eval),
            cat_features=np.arange(X.shape[1]),
            use_best_model=True,
            verbose=True)

y_pred = clf.predict(X_test).reshape(1, -1)

print("Accuracy:", np.mean(y_pred == y_sex_test))
print("Most important features:")
for i, name in enumerate(most_important_features(clf.get_feature_importance(), features, 10)):
    print(str(i+1) + ".", name)

0:	learn: 0.7487728	test: 0.7189909	best: 0.7189909 (0)	total: 21.8ms	remaining: 1m 48s
500:	learn: 0.8639551	test: 0.8556412	best: 0.8556412 (470)	total: 9.42s	remaining: 1m 24s
1000:	learn: 0.8795582	test: 0.8654520	best: 0.8654520 (787)	total: 18.5s	remaining: 1m 13s
1500:	learn: 0.8884993	test: 0.8703574	best: 0.8717589 (1417)	total: 27.8s	remaining: 1m 4s
2000:	learn: 0.8963885	test: 0.8724597	best: 0.8724597 (1914)	total: 37.1s	remaining: 55.7s
2500:	learn: 0.9055049	test: 0.8710582	best: 0.8731605 (2026)	total: 46.6s	remaining: 46.6s
bestTest = 0.8731604765
bestIteration = 2026
Shrink model to first 2027 iterations.
Accuracy: 0.8600252206809584
Most important features:
1. 40kg
2. girlmeme
3. mudakoff
4. modnailru
5. femalemem
6. thesmolny
7. reflexia_our_feelings
8. zerofat
9. rapnewrap
10. i_d_t
